# 機械学習をPythonで実践する-18　　～ 様々なエンコーディング手法 ～

In [1]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.compose import ColumnTransformer
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from lightGBM_cv import lightGBM_classifier_cv_func


%matplotlib inline
import matplotlib.pyplot as plt


## Label Encoding
ペンギンデータセットに対してラベルエンコーディングを適用する。  
このデータセットの質的変数は順序尺度ではないので、ラベルエンコーディングは適さないが、  
決定木やGBDTであればラベルエンコーディングで問題ないので、これを想定してやってみる。

In [15]:
# データセットのカラム名やデータ型は分かっているので、dtypesを指定しておく。
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

In [14]:
df.head()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
str,str,f32,f32,f32,f32,str
"""Adelie""","""Torgersen""",39.099998,18.700001,181.0,3750.0,"""MALE"""
"""Adelie""","""Torgersen""",39.5,17.4,186.0,3800.0,"""FEMALE"""
"""Adelie""","""Torgersen""",40.299999,18.0,195.0,3250.0,"""FEMALE"""
"""Adelie""","""Torgersen""",null,null,null,null,null
"""Adelie""","""Torgersen""",36.700001,19.299999,193.0,3450.0,"""FEMALE"""


In [7]:
# 量的変数と質的変数のカラム名をそれぞれ取得。今回は目的変数も含めてエンコーディングしちゃう。
num_cols = df.select(pl.col(pl.Float32)).columns
cat_cols = df.select(pl.col(pl.Utf8)).columns

In [17]:
# ラベルエンコーディング（LabelEncoderではなく、OrdinalEncoderを使う）
oe = OrdinalEncoder()
# pandasで返ってくるように指定。polarsは指定できない模様
oe.set_output(transform='pandas')
# カテゴリ変数をエンコーディング。polars.DFはそのまま入れられないのでpandasに変換する。
df = df.with_columns( pl.from_pandas(oe.fit_transform(df.select(cat_cols).to_pandas())) )

In [18]:
df

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
f64,f64,f32,f32,f32,f32,f64
0.0,2.0,39.099998,18.700001,181.0,3750.0,2.0
0.0,2.0,39.5,17.4,186.0,3800.0,1.0
0.0,2.0,40.299999,18.0,195.0,3250.0,1.0
0.0,2.0,null,null,null,null,3.0
0.0,2.0,36.700001,19.299999,193.0,3450.0,1.0
0.0,2.0,39.299999,20.6,190.0,3650.0,2.0
0.0,2.0,38.900002,17.799999,181.0,3625.0,1.0
0.0,2.0,39.200001,19.6,195.0,4675.0,2.0
0.0,2.0,34.099998,18.1,193.0,3475.0,3.0


sexカラムにあった欠損値は3.0にラベル付けされた模様。